# SIMPLE TOOL FOR CALCULATING PUT/CALL OPTIONS VALUE IN THE BINOMIAL PRICING MODEL
***

We follow the exposition in Paul Wilmott's book. We price the option using the simple parametrization in which $S_{i+1} = uS_i$ with probability $p$ and $S_{i+1} = vS_i$ with probability $(1-p)$. The option price for European options is
$$V_i = \frac{V^+ _{i+1} - V^- _{i+1}}{u-V} + \frac{1}{e^{r\delta t}}\frac{uV^- _{i+1} - vV^+ _{i+1}}{u-V} $$
while for American options
$$V_i = \text{max}\left ( \frac{V^+ _{i+1} - V^- _{i+1}}{u-V} + \frac{1}{e^{r\delta t}}\frac{uV^- _{i+1} - vV^+ _{i+1}}{u-V} , F-S_i \right)$$
where $V_{i+1} ^{\pm}$ are the option values at the next step and $F$ is the strike price. Steps are defined by $\delta t = T/N$, $i=0...N$.

In [71]:
import numpy as np
from typing import List, Dict

class OptionTypeInvalid(Exception):
    def __init__(self,  message="Option type invalid"):
        self.message = message
        super().__init__(self.message)

class OptionSpeciesInvalid(Exception):
    def __init__(self,  message="Option species invalid"):
        self.message = message
        super().__init__(self.message)

def option_payoff(option_type: 'str', asset_price: 'float', strike_price: 'float') -> float:
    """ Returns the value of this option give an asset price and strike price

    Attributes
    ----------
    option_type: str
        Either 'call' or 'put'
    asset_price: float
    strike_price: float

    Returns
    -------
    float
        The value of the option
    """
    if option_type == 'call':
        return max(0,asset_price-strike_price)
    elif option_type == 'put':
        return max(0, strike_price-asset_price)
    else:
        raise OptionTypeInvalid()
    

def option_prices(option_type: 'str', option_species : 'str', strike_price: 'float', stock_init_price: 'float', u: 'float', v: 'float', risk_free_rate: 'float' , tot_time: 'float', n_steps: 'int') -> List[Dict[float, float]]:
    """ Returns the values of the option given the option details, the risk free rate, the maturity and the number of steps in the simulation

    Attributes
    ----------
    option_type: str
        Either 'call' or 'put'
    option_species: str
        Either 'european' (default) or 'american'
    strike_price: float
    stock_init_price: float
    u: float
    v: float
    risk_free_rate: float
    tot_time: float
    n_steps: int

    Returns
    -------
    List[Dict[str, int]]
        The values of the stock, of the option payoff and of the option at time step i
    """
    step_size =  tot_time/n_steps
    n_steps += 1
    stock_prices = {i: [None for _ in range(i+1)] for i in range(n_steps)}
    option_payoffs = {i: [None for _ in range(i+1)] for i in range(n_steps)}
    option_values = {i: [None for _ in range(i+1)] for i in range(n_steps)}
    for i in range(n_steps):
        for j in range(i+1):
            stock_prices[i][j] = stock_init_price * (u**(i-j)) * (v**(j))
            option_payoffs[i][j] = option_payoff(option_type, stock_prices[i][j],strike_price)
    option_values[n_steps-1] = option_payoffs[n_steps-1]
    for i in range(n_steps):
        for j in range(n_steps-i-1):
            i_retro = n_steps-i-1
            if option_species == 'european':
                option_values[i_retro-1][j] = (option_values[i_retro][j]-option_values[i_retro][j+1])/(u-v) + (u*option_values[i_retro][j+1]-v*option_values[i_retro][j])/((u-v)*np.exp(risk_free_rate*step_size))
            elif option_species == 'american':
                option_values[i_retro-1][j] = max( (option_values[i_retro][j]-option_values[i_retro][j+1])/(u-v) + (u*option_values[i_retro][j+1]-v*option_values[i_retro][j])/((u-v)*np.exp(risk_free_rate*step_size)),
                                                  option_payoff(option_type, stock_prices[i_retro-1][j], strike_price))
            else:
                raise OptionSpeciesInvalid()
            
    return [stock_prices, option_payoffs, option_values]



Example of calculation
(different prices for european and american)

In [112]:
u = 1.2 # 1.0604
v= 0.8 # 0.9431
strike_price = 100
stock_init_price = 100
r = 0.05# 0.1 annual interest
t_tot = 2 # 1/3 # in years, so 4 months
n_steps = 2 #4 # 4 steps, so delta_t = t_tot/4 = 1 month

stock_prices, option_payoffs, option_values = option_prices(option_type='put', option_species= 'european', strike_price = strike_price, stock_init_price = stock_init_price, u=u,v=v, risk_free_rate=r, tot_time = t_tot, n_steps=n_steps)

print('Stock prices')
for step, prices in stock_prices.items():
    print(step, ':', prices)
print('Option payoffs')
for step, prices in option_payoffs.items():
    print(step, ':', prices)
print('Option values')
for step, prices in option_values.items():
    print(step, ':', prices)

Stock prices
0 : [100.0]
1 : [120.0, 80.0]
2 : [144.0, 96.0, 64.00000000000001]
Option payoffs
0 : [0]
1 : [0, 20.0]
2 : [0, 4.0, 35.999999999999986]
Option values
0 : [6.194180597610661]
1 : [1.4147530940085673, 15.122942450071378]
2 : [0, 4.0, 35.999999999999986]
